In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.callbacks import CSVLogger
import matplotlib.pyplot as plt

Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
fi = open('keptcolumns.txt', 'r')
keptcolumns = fi.read().rstrip('\n').split(', ')
keptcolumns = np.array(keptcolumns)
data = pd.read_csv('epi_r.csv', usecols = keptcolumns)
data = data[pd.notnull(data['calories'])]
data = data[pd.notnull(data['protein'])]
data = data[pd.notnull(data['fat'])]
data = data[data['calories'] != 0]
data = data[data['protein'] != 0]
data = data[data['fat'] != 0]
data = data.values
h_val = data[:, 0:6]
ingred = data[:, 6:]

In [3]:
def gen_dat(xy):
    cnt = np.sum(xy == 1)
    idx = np.where(xy == 1)
    x = np.zeros((cnt, xy.shape[0]))
    y = np.zeros((cnt, xy.shape[0]))
    for i in range(cnt):
        y_idx = idx[0][i];
        x_idx = np.setdiff1d(idx[0], y_idx)
        y[i, y_idx] = 1
        x[i, x_idx] = 1
    return x, y

In [4]:
x = np.empty((0, ingred.shape[1]))
y = np.empty((0, ingred.shape[1]))
for i in range(ingred.shape[0]):
    x_tmp, y_tmp = gen_dat(ingred[i])
    x = np.append(x, x_tmp, axis = 0)
    y = np.append(y, y_tmp, axis = 0)
print(x.shape)
print(y.shape)

(58058, 308)
(58058, 308)


In [20]:
idx = np.arange(x.shape[0])
np.random.shuffle(idx)
x = x[idx]
y = y[idx]
p = x.shape[0] * 8 // 10
x_train = x[:p, :]
y_train = y[:p, :]
x_test = x[p:, :]
y_test = y[p:, :]
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(46446, 308)
(46446, 308)
(11612, 308)
(11612, 308)


In [21]:
model1 = Sequential([Dense(200, input_shape=(308,)),
                                    LeakyReLU(),
                                    
                                    Dense(300),
                                    LeakyReLU(),
                                    
                                    Dense(300),
                                    LeakyReLU(),
                                                                                                            
                                    Dense(300),
                                    LeakyReLU(),
                                    
                                    Dense(300),
                                    LeakyReLU(),
                                    
                                    Dense(308, activation='sigmoid')
                                   ])
model1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
model1.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 200)               61800     
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 200)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 300)               60300     
_________________________________________________________________
leaky_re_lu_27 (LeakyReLU)   (None, 300)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 300)               90300     
_________________________________________________________________
leaky_re_lu_28 (LeakyReLU)   (None, 300)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 300)              

In [22]:
csv_logger = CSVLogger('training1.log')
model1.fit(x_train, y_train, epochs = 22, batch_size = 128, callbacks = [csv_logger])

Epoch 1/22
46446/46446 [==============================] - 6s 120us/step - loss: 4.8122 - categorical_accuracy: 0.0281
Epoch 2/22
46446/46446 [==============================] - 6s 120us/step - loss: 4.3589 - categorical_accuracy: 0.0497
Epoch 3/22
46446/46446 [==============================] - 5s 117us/step - loss: 4.1604 - categorical_accuracy: 0.0711
Epoch 4/22
46446/46446 [==============================] - 6s 126us/step - loss: 3.9999 - categorical_accuracy: 0.1031
Epoch 5/22
46446/46446 [==============================] - 5s 106us/step - loss: 3.8448 - categorical_accuracy: 0.1341
Epoch 6/22
46446/46446 [==============================] - 4s 92us/step - loss: 3.6976 - categorical_accuracy: 0.1557
Epoch 7/22
46446/46446 [==============================] - 4s 87us/step - loss: 3.5530 - categorical_accuracy: 0.1765
Epoch 8/22
46446/46446 [==============================] - 4s 86us/step - loss: 3.4164 - categorical_accuracy: 0.1939
Epoch 9/22
46446/46446 [==============================] - 5

In [23]:
x = ingred
calories = h_val[:, 2]
protein = h_val[:, 3]
fat = h_val[:, 4]
y = protein / (calories * fat)
print(np.where(y == 0))
idx = np.argsort(y)
idx = idx[:-250]
np.random.shuffle(idx)
x = x[idx]
y = y[idx]
maxi = np.max(y)
y = y / maxi

(array([], dtype=int64),)


In [24]:
model2 = Sequential([Dense(200, input_shape=(308,)),
                                    LeakyReLU(),
                                    
                                    Dense(500),
                                    LeakyReLU(),
                                    
                                    Dense(500),
                                    LeakyReLU(),
                    
                                    Dense(500),
                                    LeakyReLU(),
                    
                                    Dense(500),
                                    LeakyReLU(),
                    
                                    Dense(500),
                                    LeakyReLU(),
                                                        
                                    Dense(1)
                                   ])
model2.compile(optimizer='adam', loss='mean_squared_error')
model2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 200)               61800     
_________________________________________________________________
leaky_re_lu_31 (LeakyReLU)   (None, 200)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 500)               100500    
_________________________________________________________________
leaky_re_lu_32 (LeakyReLU)   (None, 500)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 500)               250500    
_________________________________________________________________
leaky_re_lu_33 (LeakyReLU)   (None, 500)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 500)              

In [25]:
csv_logger = CSVLogger('training2.log')
model2.fit(x, y, epochs = 100, batch_size=1024, callbacks=[csv_logger])

Epoch 1/100
14123/14123 [==============================] - 2s 135us/step - loss: 0.0267
Epoch 2/100
14123/14123 [==============================] - 2s 109us/step - loss: 0.0170
Epoch 3/100
14123/14123 [==============================] - 2s 110us/step - loss: 0.0154
Epoch 4/100
14123/14123 [==============================] - 2s 110us/step - loss: 0.0143
Epoch 5/100
14123/14123 [==============================] - 2s 121us/step - loss: 0.0131
Epoch 6/100
14123/14123 [==============================] - 2s 114us/step - loss: 0.0120
Epoch 7/100
14123/14123 [==============================] - 2s 112us/step - loss: 0.0111
Epoch 8/100
14123/14123 [==============================] - 2s 116us/step - loss: 0.0097
Epoch 9/100
14123/14123 [==============================] - 2s 117us/step - loss: 0.0088
Epoch 10/100
14123/14123 [==============================] - 2s 128us/step - loss: 0.0082
Epoch 11/100
14123/14123 [==============================] - 2s 117us/step - loss: 0.0076
Epoch 12/100
14123/14123 [====

14123/14123 [==============================] - 2s 120us/step - loss: 0.0030
Epoch 94/100
14123/14123 [==============================] - 2s 119us/step - loss: 0.0029
Epoch 95/100
14123/14123 [==============================] - 2s 119us/step - loss: 0.0029
Epoch 96/100
14123/14123 [==============================] - 2s 119us/step - loss: 0.0029
Epoch 97/100
14123/14123 [==============================] - 2s 120us/step - loss: 0.0030
Epoch 98/100
14123/14123 [==============================] - 2s 120us/step - loss: 0.0030
Epoch 99/100
14123/14123 [==============================] - 2s 119us/step - loss: 0.0030
Epoch 100/100
14123/14123 [==============================] - 2s 120us/step - loss: 0.0030


In [43]:
cnt1 = 0
cnt2 = 0
for i in range(x_test.shape[0]):
    x = x_test[i].reshape(1, x_test.shape[1])
    y = model1.predict(x, batch_size = 128)
    y_idx = np.argsort(y[0])
    y_idx = y_idx[-10:]
    if np.where(y_test[i] == 1) in y_idx:
        cnt1 += 1
    cnt2 += 1
print(cnt1 / cnt2)

0.5320358250086118


In [44]:
cnt1 = 0
cnt2 = 0
for i in range(x_train.shape[0]):
    x = x_train[i].reshape(1, x_train.shape[1])
    y = model1.predict(x, batch_size = 128)
    y_idx = np.argsort(y[0])
    y_idx = y_idx[-10:]
    if np.where(y_train[i] == 1) in y_idx:
        cnt1 += 1
    cnt2 += 1
print(cnt1 / cnt2)

0.9109288205658184


In [62]:
def test_model(x_idx):
    x = np.zeros((1, ingred.shape[1]))
    x[0, x_idx] = 1
    y = model1.predict(x, batch_size = 128)
    y_idx = np.argsort(y[0])
    y_idx = y_idx[-10:]
    print(keptcolumns[y_idx + 6])
    print(y[0][y_idx])
    h_val = np.zeros(10)
    for i in range(10):
        x_tmp = np.append(x_idx, y_idx[i])
        x = np.zeros((1, 308))
        x[0, x_tmp] = 1        
        y_tmp = model2.predict(x, batch_size = 128)
        h_val[i] = y_tmp
    y_idx_idx = np.argsort(h_val)
    y_idx = y_idx[y_idx_idx]
    print(keptcolumns[y_idx + 6])
    print(h_val[y_idx_idx])

In [63]:
rec_no = 15
print(h_val[rec_no][0])
idx = np.array(np.where(ingred[rec_no] == 1))
print(idx)
print(keptcolumns[idx + 6])
print('Suggestions:')
test_model(np.array([8, 46, 219, 230]));

Tuna, Asparagus, and New Potato Salad with Chive Vinaigrette and Fried Capers 
[[  8  46 219 230 289]]
[['asparagus' 'capers' 'potato' 'radish' 'tuna']]
Suggestions:
['mushroom' 'onion' 'olive' 'fennel' 'bean' 'green bean' 'cauliflower'
 'chicken' 'egg' 'tuna']
[0.01371816 0.01823738 0.02338681 0.02453548 0.0288403  0.03039324
 0.0519034  0.74252915 0.9438875  0.9443097 ]
['green bean' 'chicken' 'tuna' 'bean' 'onion' 'olive' 'mushroom' 'egg'
 'cauliflower' 'fennel']
[0.02723945 0.04097425 0.04493065 0.05182261 0.05846273 0.06598619
 0.08816563 0.08823858 0.09497377 0.09989666]
